# Procept dashboard

In [3]:
import pandas as pd
from sqlalchemy import text, create_engine

In [36]:
#Hidden connection to database

## Creating the main tables for the dashboard:

### 1. Inventory Stock Table

**Main SQL request:**

In [19]:
wharehouse = '''
SELECT ms.quantity, ms.product_id, date, wh.name AS wh_name, wh.type, mp.name AS mp_name, 
mpr.id AS mpr_id, mpr.article as product_article, mpr.name AS product_name, 
mcat.name AS category, midcat.name AS mid_category, topcat.name AS top_category

FROM marketing_productstock ms

LEFT JOIN marketing_warehouse AS wh ON ms.warehouse_id = wh.id
LEFT JOIN marketing_marketplace AS mp ON wh.marketplace_id = mp.id
LEFT JOIN marketing_product AS mpr ON ms.product_id = mpr.id
LEFT JOIN marketing_productcategory mcat ON mpr.category_id = mcat.id
LEFT JOIN marketing_middlelevelcategory midcat ON mcat.parent_id = midcat.id
LEFT JOIN marketing_toplevelcategory topcat ON midcat.parent_id = topcat.id
'''
con=engine.connect()
wharehouse = pd.io.sql.read_sql(sql=text(wharehouse), con = con)

**Check data in tables:**

In [5]:
marketing_product = '''SELECT * FROM marketing_product'''
con=engine.connect()
marketing_product = pd.io.sql.read_sql(sql=text(marketing_product), con = con)

In [6]:
marketing_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         551 non-null    int64  
 1   article                    551 non-null    object 
 2   ean_13                     523 non-null    object 
 3   name                       551 non-null    object 
 4   cost                       549 non-null    float64
 5   min_recommended_price      547 non-null    float64
 6   recommended_price          547 non-null    float64
 7   category_id                541 non-null    float64
 8   ozon_name                  514 non-null    object 
 9   expiration_date            1 non-null      float64
 10  group_barcode              0 non-null      object 
 11  quantity_in_group_package  1 non-null      float64
 12  name_1c                    543 non-null    object 
 13  wb_name                    476 non-null    object 

In [8]:
print('All SKU of the supplier:', marketing_product['article'].count())

All SKU of the supplier: 551


In [10]:
marketing_productstock = '''SELECT * FROM marketing_productstock'''
con=engine.connect()
marketing_productstock = pd.io.sql.read_sql(sql=text(marketing_productstock), con = con)
marketing_productstock.head()

,id,quantity,product_id,warehouse_id,date
0,24023,0,241,30,2023-06-25 21:00:00+00:00
1,24024,0,254,30,2023-06-25 21:00:00+00:00
2,24025,0,258,30,2023-06-25 21:00:00+00:00
3,24026,0,245,30,2023-06-25 21:00:00+00:00
4,24027,0,320,30,2023-06-25 21:00:00+00:00


In [73]:
marketing_productstock['product_id'].value_counts()

195    902
157    902
159    902
55     902
156    902
      ... 
482     23
15       6
191      2
189      2
190      2
Name: product_id, Length: 396, dtype: int64

In [11]:
print('Number of warehouses Waldberries',  marketing_productstock['warehouse_id'].nunique())

Number of warehouses: 40


In [12]:
marketing_warehouse = '''SELECT * FROM marketing_warehouse'''
con=engine.connect()
marketing_warehouse = pd.io.sql.read_sql(sql=text(marketing_warehouse), con = con)
marketing_warehouse.head()

,id,name,type,city,marketplace_id
0,7,FRESH_MAX_МСК_МИТИНО,FBO,None,2
1,8,Хабаровск_РФЦ,FBO,None,2
2,9,Тверь_Хаб,FBO,None,2
3,10,ЖУКОВСКИЙ_РФЦ,FBO,None,2
4,11,Хоругвино_РФЦ_НЕГАБАРИТ,FBO,None,2


In [66]:
marketing_warehouse['name'].nunique()

58

In [14]:
marketing_marketplace = '''SELECT * FROM marketing_marketplace'''
con=engine.connect()
marketing_marketplace = pd.io.sql.read_sql(sql=text(marketing_marketplace), con = con)
marketing_marketplace

,id,name
0,1,Wildberries
1,2,Ozon
2,3,YM
3,4,WB_SPP
4,5,Wildberries_price
5,9,Ozon_SPP
6,10,Ozon_GREEN


In [16]:
print('Marketplace name:', marketing_marketplace['name'].unique())

Marketplace name: ['Wildberries' 'Ozon' 'YM' 'WB_SPP' 'Wildberries_price' 'Ozon_SPP'
 'Ozon_GREEN']


In [22]:
avg_whareh = '''

SELECT *
FROM 
(
SELECT CAST(t1.date AS DATE) AS date, sum(t1.quantity) AS whareh
FROM
(
SELECT ms.quantity, ms.product_id, date, wh.name AS wh_name, wh.type, mp.name AS mp_name, 
mpr.id AS mpr_id, mpr.article as product_article, mpr.name AS product_name, 
mcat.name AS category, midcat.name AS mid_category, topcat.name AS top_category
FROM marketing_productstock ms
LEFT JOIN marketing_warehouse AS wh ON ms.warehouse_id = wh.id
LEFT JOIN marketing_marketplace AS mp ON wh.marketplace_id = mp.id
LEFT JOIN marketing_product AS mpr ON ms.product_id = mpr.id
LEFT JOIN marketing_productcategory mcat ON mpr.category_id = mcat.id
LEFT JOIN marketing_middlelevelcategory midcat ON mcat.parent_id = midcat.id
LEFT JOIN marketing_toplevelcategory topcat ON midcat.parent_id = topcat.id
) AS t1
GROUP BY date
ORDER BY date ASC NULLS FIRST) as t2
GROUPBY
'''
con=engine.connect()
avg_whareh = pd.io.sql.read_sql(sql=text(avg_whareh), con = con)
avg_whareh

,date,whareh
0,2023-06-26,150805
1,2023-06-27,64724
2,2023-06-28,212140
3,2023-06-29,212074
4,2023-06-30,222417
5,2023-07-03,349742
6,2023-07-04,220527
7,2023-07-05,220086
8,2023-07-06,219837
9,2023-07-07,279903


In [70]:
left.groupby(['mp_name', 'wh_name']).agg({'quantity':'sum'})

quantity
mp_name     wh_name                                        
Ozon        FRESH_MAX_МСК_АВТОГРАД                        0
            FRESH_MAX_МСК_ЕРЕМИНО                         0
            FRESH_MAX_МСК_КРАСНАЯ_ПРЕСНЯ                  0
            FRESH_MAX_МСК_МИТИНО                          0
            FRESH_MAX_МСК_ПОДОЛЬСК                        0
            FRESH_MAX_МСК_ПРИВОЛЬНАЯ                      0
            FRESH_MAX_СПБ_ЛИГОВСКИЙ                       0
            FRESH_MAX_СПБ_ЛОГОПАРК                        0
            FRESH_MAX_СПБ_РАУМ_БУГРЫ                      0
            АДЫГЕЙСК_РФЦ                               3098
            ВОРОНЕЖ_МРФЦ                                 54
            ВолхонскоеFBS                            739656
            Дер. ГрибкиFBS                          1374158
            Екатеринбург_РФЦ_НОВЫЙ                     1772
            ЖУКОВСКИЙ_РФЦ                                51
            КАЛИНИНГРАД_МРФЦ                            473
            КРАСНОЯРСК_МРФЦ                             449
            Казань_РФЦ_НОВЫЙ                           2664
            НИЖНИЙ_НОВГОРОД_РФЦ                           8
            НОВОРОССИЙСК_МРФЦ                            59
            Новая_Рига_РФЦ                                0
            Новосибирск_РФЦ_НОВЫЙ                       226
            ПАВЛО_СЛОБОДСКОЕ_РФЦ_НЕГАБАРИТ                0
            ПУШКИНО_1_РФЦ                             63068
            ПУШКИНО_2_РФЦ                               177
            Ростов_на_Дону_РФЦ                           76
            САМАРА_РФЦ                                   11
            СОФЬИНО_РФЦ                                   0
            СПБ_БУГРЫ_РФЦ                                12
            Санкт_Петербург_РФЦ                       50672
            ТВЕРЬ_РФЦ                                   989
            Тверь_Хаб                                     0
            ХОРУГВИНО_РФЦ                             47549
            Хабаровск_РФЦ                               114
            Хоругвино_РФЦ_НЕГАБАРИТ                     569
Wildberries ВолхонскоеwbFBS                               0
            ГрибкиwbFBS                             1347253
YM          ЛО Парголово_YM                           10100
            Мытищи, Ангарская, 40, 7_YM              690224
            Санкт-Петербург,  Волхонское шоссе_YM    867215

In [79]:
marketing_productstock['date'].min()

Timestamp('2023-06-25 21:00:00+0000', tz='UTC')

In [78]:
marketing_productstock['date'].max()

Timestamp('2023-08-27 21:00:00+0000', tz='UTC')

### 2.  Aggregated  Product Sales

**Main SQL request:**

In [21]:
agg_sales = '''
with 
profit AS (SELECT mr.shk_id, mr.sale_dt , mr.subject_name , mr.nm_id, mr.sa_name, mr.quantity ,
mr.order_id, mr.ppvz_for_pay AS income, mr.retail_price AS retail_price,  mr.retail_amount AS retail_amount
FROM marketing_realizationreport AS mr
WHERE mr.supplier_oper_name LIKE '%Продажа%'),

returned AS (SELECT shk_id, sale_dt,  ppvz_for_pay AS returned
FROM marketing_realizationreport
WHERE supplier_oper_name LIKE '%Возврат%'), 

returned_storno AS (SELECT shk_id, sale_dt,  ppvz_for_pay AS returned_storno
FROM marketing_realizationreport
WHERE supplier_oper_name LIKE '%Сторно возвратов%'), 

sold_storno AS (SELECT shk_id, sale_dt,  ppvz_for_pay AS sold_storno
FROM marketing_realizationreport
WHERE supplier_oper_name LIKE '%Сторно продаж%'),

delivery AS (SELECT shk_id,  SUM(delivery_rub) AS delivery
FROM marketing_realizationreport
WHERE supplier_oper_name LIKE '%Логистика%' AND shk_id >0
GROUP BY shk_id), 
--тут есть ноль в штрих коде

delivery_storno AS (SELECT shk_id, sale_dt,  delivery_rub AS delivery_storno
FROM marketing_realizationreport
WHERE supplier_oper_name LIKE '%Логистика сторно%'), 

penalty AS (SELECT shk_id,  SUM(penalty) AS penalty
FROM marketing_realizationreport
WHERE supplier_oper_name LIKE '%Штрафы%' AND shk_id >0
GROUP BY shk_id),


agregated_table AS
(SELECT p.shk_id, p.sale_dt, p.subject_name, p.nm_id, p.sa_name, p.quantity, p.order_id, p.retail_price, p.retail_amount, p.income, r.returned, returned_storno,
sold_storno, delivery, delivery_storno, penalty
FROM profit as p
LEFT JOIN returned AS r ON (p.shk_id = r.shk_id) AND (p.sale_dt = r.sale_dt)
LEFT JOIN returned_storno AS rs ON (p.shk_id = rs.shk_id) AND (p.sale_dt = rs.sale_dt)
LEFT JOIN sold_storno AS ss ON (p.shk_id = ss.shk_id) AND (p.sale_dt = ss.sale_dt)
LEFT JOIN delivery AS d ON p.shk_id = d.shk_id
LEFT JOIN delivery_storno AS ds ON p.shk_id = ds.shk_id
LEFT JOIN penalty AS pn ON p.shk_id = pn.shk_id)


SELECT mr.shk_id, mr.sale_dt, mr.subject_name, mr.nm_id, mr.sa_name, mr.quantity, mr.order_id ,mr.retail_price, mr.retail_amount, mr.income, mr.returned, mr.returned_storno,
mr.sold_storno, mr.delivery, mr.delivery_storno, mr.penalty,
mpr.id AS product_id, mpr.name AS product_name, mpr.cost , mpr.category_id,
mcat.name AS category,
midcat.name AS mid_category,
topcat.name AS top_category

FROM agregated_table AS mr
LEFT JOIN marketing_product AS mpr ON mr.sa_name = mpr.article
LEFT JOIN marketing_productcategory mcat ON mpr.category_id = mcat.id
LEFT JOIN marketing_middlelevelcategory midcat ON mcat.parent_id = midcat.id
LEFT JOIN marketing_toplevelcategory topcat ON midcat.parent_id = topcat.id

'''
con=engine.connect()
agg_sales = pd.io.sql.read_sql(sql=text(agg_sales), con = con)
agg_sales.head()

,shk_id,sale_dt,subject_name,nm_id,sa_name,quantity,order_id,retail_price,retail_amount,income,...,delivery,delivery_storno,penalty,product_id,product_name,cost,category_id,category,mid_category,top_category
0,11418845127,2023-05-11 00:00:00+00:00,Краски водно-дисперсионные,151570862,079-1,1,2344.0,373.0,354.0,272.78,...,144.15,NaN,3038.0,228,Краска для плит OSB для внутренних и наружных ...,179.0,30,"Краска, эмаль хозяйственные",Краски/Лаки/Масла,Стройка
1,10625550341,2023-04-12 00:00:00+00:00,Лаки строительные,150031320,053-09,1,NaN,832.0,584.0,564.71,...,233.50,NaN,NaN,323,"Лак для камня PROSEPTготовый состав / 0,9 л",287.0,31,Лак хозяйственный,Краски/Лаки/Масла,Стройка
2,11030796648,2023-04-19 00:00:00+00:00,Антисептики-пропитки,150030037,077-5,1,1490.0,5790.0,3658.0,3572.05,...,145.47,NaN,NaN,215,Антисептик усиленный невымываемыйPROSEPT ULTRA...,1989.0,20,Антисептик строительный,Деревообработка,Стройка
3,10844006926,2023-04-15 00:00:00+00:00,Антисептики-пропитки,150123389,075-16,1,1293.0,6823.0,4797.0,4774.69,...,535.24,NaN,NaN,311,Сухой концентрат огнебиозащита мешок / 16 кг,2873.0,38,Пропитка строительная (Дерево),Деревообработка,Стройка
4,11364563628,2023-05-12 00:00:00+00:00,Герметики,150126204,0024-7с,1,2576.0,6387.0,2557.0,2430.90,...,184.32,NaN,NaN,371,"Герметик акриловый цвет Сосна, 7 кг",2646.0,25,Герметик строительный,Деревообработка,Стройка


**Check data in tables:**

In [23]:
realization = '''SELECT * FROM marketing_realizationreport'''
con=engine.connect()
realization = pd.io.sql.read_sql(sql=text(realization), con = con)

In [25]:
realization_1 = realization[['sale_dt', 'subject_name', 'nm_id', 'sa_name', 'quantity', 'supplier_oper_name', 'ppvz_for_pay', 'penalty', 'additional_payment', 'delivery_rub', 'order_id', 'product_id']]
realization_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42982 entries, 0 to 42981
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   sale_dt             37444 non-null  datetime64[ns, UTC]
 1   subject_name        42982 non-null  object             
 2   nm_id               42982 non-null  int64              
 3   sa_name             42982 non-null  object             
 4   quantity            42982 non-null  int64              
 5   supplier_oper_name  42982 non-null  object             
 6   ppvz_for_pay        42982 non-null  float64            
 7   penalty             42982 non-null  float64            
 8   additional_payment  42982 non-null  float64            
 9   delivery_rub        37444 non-null  float64            
 10  order_id            22425 non-null  float64            
 11  product_id          42933 non-null  float64            
dtypes: datetime64[ns, UTC](1), float

In [27]:
realization_1.query('sale_dt.isnull()').head()

,sale_dt,subject_name,nm_id,sa_name,quantity,supplier_oper_name,ppvz_for_pay,penalty,additional_payment,delivery_rub,order_id,product_id
54,NaT,Шампуни,150127175,С001-5,1,Возмещение издержек по перевозке,0.0,0.0,0.0,NaN,NaN,441.0
151,NaT,Антисептики-пропитки,150030036,077-1,1,Возмещение издержек по перевозке,0.0,0.0,0.0,NaN,NaN,214.0
234,NaT,Очистители плесени,155193278,019-05,1,Возмещение издержек по перевозке,0.0,0.0,0.0,NaN,NaN,390.0
288,NaT,Спреи для уборки,149419143,304-05,1,Возмещение издержек по перевозке,0.0,0.0,0.0,NaN,NaN,16.0
357,NaT,Спреи для уборки,150008735,257-05,1,Возмещение издержек по перевозке,0.0,0.0,0.0,NaN,NaN,18.0


In [29]:
print('nm_id:',realization_1['nm_id'].nunique())

nm_id: 301


In [31]:
print('wb_article:',marketing_product['wb_article'].nunique())

wb_article: 351


In [32]:
realization_1['supplier_oper_name'].unique()

array(['Продажа', 'Логистика', 'Частичная компенсация брака',
       'Возмещение издержек по перевозке', 'Штрафы', 'Возврат',
       'Сторно возвратов', 'Логистика сторно',
       'Авансовая оплата за товар без движения', 'Сторно продаж'],
      dtype=object)

Check the overall change in sales growth for the entire period:

In [34]:
total_sales_growth = '''
WITH 
t AS
(SELECT CAST(mr.sale_dt AS date) ,
COUNT(supplier_oper_name) AS purchase,
LAG(COUNT(supplier_oper_name)) OVER(ORDER BY CAST(mr.sale_dt AS date)) prev_purchase
FROM marketing_realizationreport AS mr
WHERE supplier_oper_name LIKE '%Продажа%'
GROUP BY CAST(mr.sale_dt AS date)
ORDER BY CAST(mr.sale_dt AS date) ASC)

SELECT (((CAST(purchase AS FLOAT)) - (CAST(prev_purchase AS FLOAT)))/(CAST(prev_purchase AS FLOAT))*100) as sales_growth_idx
FROM t
ORDER BY sale_dt DESC
LIMIT 1

'''
con=engine.connect()
total_sales_growth = pd.io.sql.read_sql(sql=text(total_sales_growth), con = con)
total_sales_growth

,sales_growth_idx
0,37.383178
